In [1]:
import tensorflow as tf

In [6]:
#constant is type of node.
node1 = tf.constant(13)
print(node1)
node2 = tf.constant([[1,1.2],[21,45],[4,56]])
print(node2)

Tensor("Const_2:0", shape=(), dtype=int32)
Tensor("Const_3:0", shape=(3, 2), dtype=float32)


In [3]:
#create a session object and invoke run method to run the computational graph
sess = tf.Session()
print(sess.run(node1))
print(sess.run(node2))
print(sess.run([node1, node2]))

13
[[ 1 12]
 [21 45]
 [ 4 56]]
[13, array([[ 1, 12],
       [21, 45],
       [ 4, 56]], dtype=int32)]


In [4]:
node3 = tf.add(node1, node2)
print(sess.run(node3))

[[14 25]
 [34 58]
 [17 69]]


In [5]:
#a graph can be paramtererized to accept external inputs called placeholders
a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)

#operation to be performed
adder_node = a + b

'''
We can evaluate this graph with multiple inputs by using the 
feed_dict argument to the run method to feed concrete values to the placeholders
'''

print(sess.run(adder_node, {a: 3, b: 4}))
print(sess.run(adder_node, {a:[[2,3,4], [23, 43, 2]], b: [[10], [12]]}))


7.0
[[ 12.  13.  14.]
 [ 35.  55.  14.]]


In [7]:
'''
to train a model, we need to be able to modify the graph to get new outputs with same input
Variables allow us to add trainable parameters to a graph
They're constructed with a type and initial value
'''
W = tf.Variable([.3], dtype=tf.float32)
b = tf.Variable([-.3], dtype=tf.float32)
x = tf.placeholder(tf.float32)

linear_model = W*x + b


In [8]:
'''
By contrast, variables are not initialized when you call tf.Variable. 
To initialize all the variables in a TensorFlow program, 
you must explicitly call global_variables_initializer()
'''
init = tf.global_variables_initializer()
sess.run(init)
'''
init is a handle to the TensorFlow sub-graph that initializes all the global variables. 
Until we call sess.run, the variables are uninitialized.
'''
print(sess.run(linear_model, {x: [1, 2, 3, 4]}))

[ 0.          0.30000001  0.60000002  0.90000004]


In [11]:
'''
We've created a model, but we don't know how good it is yet. 
To evaluate the model on training data, 
we need a y placeholder to provide the desired values, 
and we need to write a loss function.
A loss function measures how far apart the current model is from the provided data.
linear_model - y
'''

y = tf.placeholder(tf.float32)
squared_deltas = tf.square(linear_model - y)
loss = tf.reduce_sum(squared_deltas)

print("squared detlas:", sess.run(squared_deltas, {x: [1, 2, 3, 4], y:[0, -1, -2, -3]}))
print("loss:", sess.run(loss, {x: [1, 2, 3, 4], y:[0, -1, -2, -3]}))


('squared detlas:', array([  0.        ,   1.68999982,   6.75999928,  15.21000099], dtype=float32))
('loss:', 23.66)


In [12]:
'''
A variable is initialized to the value provided to tf.Variable 
but can be changed using operations like tf.assign
'''
fixW = tf.assign(W, [-1.])
fixb = tf.assign(b, [1.])
sess.run([fixW, fixb])
print(sess.run(loss, {x: [1, 2, 3, 4], y: [0, -1, -2, -3]}))

0.0


In [13]:
'''
TensorFlow provides optimizers that slowly change each variable in order to minimize the loss function. 
The simplest optimizer is gradient descent
ensorFlow produces derivatives given only a description of the model using the function tf.gradients
'''
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

sess.run(init)# reset values to incorrect defaults
for i in range(1000):
    sess.run(train, {x: [1, 2, 3, 4], y: [0, -1, -2, -3]})

sess.run([W, b])

[array([-0.9999969], dtype=float32), array([ 0.99999082], dtype=float32)]